In [ ]:
%pip install transformers torch
%pip install --upgrade pip setuptools wheel


  Using cached transformers-4.52.3-py3-none-any.whl.metadata (40 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
Using cached transformers-4.52.3-py3-none-any.whl (10.5 MB)
Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl (418 kB)
Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.5 MB/s eta 0:00:00a 0:00:01
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
Note: you may need to restart the kerne

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from datasets import load_dataset
from tqdm import tqdm
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_and_prepare_data(dataset_name="gsarti/flores_101", language="eng", split="devtest"):
    """Load and prepare the Flores dataset."""
    try:
        dataset = load_dataset(dataset_name, name=language, split=split, trust_remote_code=True)
        return [{"eng": entry["sentence"]} for entry in dataset]
    except Exception as e:
        logger.error(f"Error loading dataset: {e}")
        raise

def initialize_model(model_name="facebook/nllb-200-distilled-600M"):
    """Initialize and return the translation model and tokenizer."""
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        return tokenizer, model, device
    except Exception as e:
        logger.error(f"Error initializing model: {e}")
        raise

def translate_english_to_zulu(ref_sentences, output_file="flores101.hyp.txt"):
    """
    Translate English sentences to Zulu using NLLB-200 model.
    
    Args:
        ref_sentences: List of dictionaries with "eng" keys containing sentences
        output_file: Path to save the translations
    """
    try:
        tokenizer, model, device = initialize_model()
        target_lang = "zul_Latn"  # Zulu language code for NLLB
        
        # Get the target language token ID
        forced_bos_token_id = tokenizer.convert_tokens_to_ids(target_lang)
        
        logger.info(f"Starting translation of {len(ref_sentences)} sentences...")
        
        with open(output_file, "w", encoding="utf-8") as f:
            for entry in tqdm(ref_sentences, desc="Translating"):
                # Tokenize the input text
                inputs = tokenizer(
                    entry["eng"],
                    return_tensors="pt",
                    padding=True,
                    truncation=True
                ).to(device)
                
                # Generate translation with target language specification
                with torch.no_grad():
                    translated = model.generate(
                        **inputs,
                        forced_bos_token_id=forced_bos_token_id
                    )
                
                decoded = tokenizer.decode(translated[0], skip_special_tokens=True)
                f.write(decoded.strip() + "\n")
        
        logger.info(f"Translation completed. Results saved to {output_file}")
    except Exception as e:
        logger.error(f"Error during translation: {e}")
        raise

if __name__ == "__main__":
    # Load data
    ref_sentences = load_and_prepare_data()
    
    # Perform translation
    translate_english_to_zulu(ref_sentences)


INFO:__main__:Starting translation of 1012 sentences...
Translating: 100%|██████████| 1012/1012 [19:10<00:00,  1.14s/it]
INFO:__main__:Translation completed. Results saved to flores101.hyp.txt
